# Стохастичний градієнтний спуск

&emsp;&emsp;Для прикладу розглянемо задачу оптимізації, яку можна вирішити методом градієнтного спуску: знайти таке значення $\mu$, при якому наступна функція набуває мінімального значення:

$$f = \frac{1}{2} \sum_{i=1}^{n}{(x_{i} - \mu)^{2}}$$

&emsp;&emsp;Запишемо похідну та вирішимо задачу звичайним методом градієнтного спуску для деякого набору даних

$$\frac{df}{d \mu} = -\sum_{i=1}^{n}{(x_{i} - \mu)}$$

In [1]:
import numpy as np
from matplotlib import pyplot as plt

X = np.random.normal(2, 1, size=10_000)  # згенерируємо дані

# визначимо цільову функцію задачі
def f(x, m):
    return 0.5 * np.sum((x - m) ** 2)

# визначимо похідну цільової функції
def df(x, m):
    return -np.sum(x - m)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
mu = 10  # початкова гіпотеза про значення параметра
alpha = 0.00001  # параметр швидкості навчання
n_iterations = 100  # кількість кроків алгоритму (ітерацій)

mu_values = [mu]  # список, до якого будемо записувати проміжні значення
for i in range(n_iterations):
    mu -= alpha * df(X, mu)  # оновлення параметра
    mu_values.append(mu)

print(mu)
# графік процесу пошуку параметра
plt.plot(range(n_iterations + 1), mu_values)
plt.show()

&emsp;&emsp;Запропонований вище метод призводить до правильного рішення. Однак, зверніть увагу, що щоразу при знаходженні похідної нам необхідно підсумовувати всі значення Х. Ми можемо собі це дозволити, якщо набір даних не надто великий і якщо функція похідної не дуже складна, проте, якби у нас були мільярди записів для Х, то ми фізично не змогли цього зробити, оскільки всі значення не помістилися б у пам'яті комп'ютера.

&emsp;&emsp;Модифікація методу, яка називається стохастичний градієнтний спуск, полягає в наступному:

&emsp;&emsp;1. Розіб'єм дані на невеликі фрагменти, які помістяться в пам'яті (наприклад, по 64 спостереження за раз).

&emsp;&emsp;2. Обчислимо значення похідної лише поточного маленького фрагмента даних (такий фрагмент зазвичай називається batch чи mini batch) і оновимо значення параметра користуючись лише цією похідною.

&emsp;&emsp;3. Перейдемо до наступного батчу та повторимо процедуру. Одне оновлення називається ітерацією
(ітерація чи наближення - у випадку, це один крок, у якому відбувається оновлення параметрів алгоритму). Цикл, у якому ми послідовно перебрали всі дані, які ми маємо, називатимемо епохою.

In [ ]:
mu = 10  # початкова гіпотеза про значення параметра
alpha = 0.00001  # ппараметр швидкості навчання
n_epochs = 100  # кількість епох: скільки разів алгоритм "побачить" всі дані
batch_size = 64  # розмір батча

mu_values = [mu]
for epoch in range(n_epochs):
    i = 0
    # пройдемося по всіх даним порційно
    while i + batch_size < len(X):
        
        # сформуємо поточний батч
        batch = X[i:i+batch_size]
        
        # оновимо значення параметра, використовуючи тільки дані батча
        mu -= alpha * df(batch, mu)
        i += batch_size
    
    mu_values.append(mu)

plt.plot(range(n_epochs + 1), mu_values)
plt.show()

&emsp;&emsp;Як бачимо, ми отримали той самий результат, але шляхом, що вимагає набагато менше пам'яті одночасно. У разі величезних обсягів даних ми таким чином можемо зчитувати їх по 64 спостереження прямо з диска і видаляти їх із пам'яті після обробки.

&emsp;&emsp;Стохастичний градієнтний спуск (та його модифікації) - основний алгоритм навчання нейронних мереж.


# Перцептрон

&emsp;&emsp;Базовою одиницею нейронної мережі є перцептрон. Зобразити цей блок графічно можна так:

![](Single_layer_perceptron.jpg)

&emsp;&emsp;По-суті, це картинка моделі логістичної регресії. Тут $x_{i}$ - назва змінної, $w_{i}$ - вага моделі, на яку домножується кожна змінна.
Червоний круг відповідає підсумовуванню, тобто це скалярний добуток $w^{T}x$, а $f(x)$ - логістична (чи якась інша) функція.

&emsp;&emsp;Вирішимо задачу класифікації ірисів, використовуючи логістичну регресію у вигляді запропонованого вище персептрону

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# вивантажимо дані ірисів
iris = datasets.load_iris()
X, y = iris.data, iris.target

# візьмемо лише 2 класи і розіб'ємо на навчальну та тестову групи
X, y = X[y >= 1], y[y >= 1]
y = np.where(y == 2, 1, 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
# pip install keras

In [ ]:
# pip install tensorflow

&emsp;&emsp;Тепер ініціалізуємо модель

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# фіксація випадкового стану
np.random.seed(1)
tf.random.set_seed(1)

model = Sequential()  # ініціалізація порожньої моделі

# додамо один перцептрон у модель
model.add(Dense(
    units=1,  # кількість перцептронів
    activation='sigmoid'  # функція активації: перетворення на виході моделі
))


&emsp;&emsp;На наступному кроці необхідно визначитися з алгоритмом навчання та скомпілювати модель. Як було сказано вище, основним алгоритмом навчання нейронних мереж є стохастичний градієнтний спуск, однак у нього є кілька варіацій з додатковими параметрами. Скористаємося найпростішим варіантом, імплементованим у keras

In [ ]:
from keras.optimizers import SGD

model.compile(
    optimizer=SGD(learning_rate=0.01),  # алгоритм оптимізації та швидкість навчання
    loss='binary_crossentropy',  # функція втрат (та сама, що в логістичної регресії)
    metrics=['accuracy']  # додаткова метрика якості моделі
)

&emsp;&emsp;Тепер можемо приступати до навчання

In [ ]:
history = model.fit(
    x=X_train, y=y_train,  # навчальні дані
    batch_size=8,  # розмір батча
    epochs=300,  # кількість епох навчання
    validation_data=(X_test, y_test)  # дані для валідації
)

&emsp;&emsp;Визначимо функцію, яка будує графік зміни помилки на навчальній та тестовій групі залежно від епохи та подивимося, як проходив процес навчання

In [ ]:
def plot_model_history(hist):
    train_loss, test_loss = hist.history['loss'], hist.history['val_loss']
    x = np.arange(len(train_loss))
    plt.plot(x, train_loss, label='Train loss')
    plt.plot(x, test_loss, label='Test loss')
    plt.legend()
    plt.show()
    
plot_model_history(history)

&emsp;&emsp;На відміну від логістичної регресії, навчання нейронної мережі не зупиниться самостійно. Його прийнято переривати на тому моменті, коли спостерігається погіршення точності на тестовому наборі (так само, як і для градієнтного бустингу).

&emsp;&emsp;Розглянемо більш цікаву задачу, на яку логістична регресія неспроможна дати задовільного рішення: завдання виключаючого або (XOR). Для цього згереруємо двовимірні дані, кожен вимір яких може набувати від'ємних і додатніх значень. Якщо обидва виміри додатні чи обидва від'ємні, то точка належить одному класу, інакше - іншому.

In [ ]:
np.random.seed(42)

X = np.random.randn(300, 2)  # згенеруємо дані
y = np.array(np.logical_xor(X[:, 0] > 0, X[:, 1] > 0), dtype=int)  # створимо класи

# відобразимо отримані класи на графіку
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

&emsp;&emsp;Спробуємо застосувати до даних ту саму нейронну мережу, яка імітує модель логістичної регресії:

In [ ]:
# фіксація випадкового стану
np.random.seed(1)
tf.random.set_seed(1)

model = Sequential()  # ініціалізація порожньої моделі

# додамо один персептрон у модель
model.add(Dense(
    units=1,  # кількість перцептронів
    activation='sigmoid'  # функція активації: перетворення на виході моделі
))

model.compile(
    optimizer=SGD(learning_rate=0.01),  # алгоритм оптимізації та швидкість навчання
    loss='binary_crossentropy',  # функція втрат (та сама, що в логістичної регресії)
    metrics=['accuracy']  # додаткова метрика якості моделі
)

history = model.fit(x=X, y=y, batch_size=8, epochs=50)


&emsp;&emsp;Як бачимо, точність залишається близько 50% скільки б модель не навчалася. Це пов'язано з тим, що межа прийняття рішень в логістичної регресії - пряма лінія, а наші дані неможливо розділити однією прямою.

In [ ]:
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(X, y, clf=model)
plt.show()

&emsp;&emsp;Щоб впоратися з таким завданням необхідно вивчити більш складну функцію прийняття рішень. Цього можна досягти, якщо додати прихований шар у нашу модель:

![](maxresdefault.jpg)

&emsp;&emsp;На картинці вище кожен синій круг - це змінна, яка надходить на вхід моделі. Кожен фіолетовий круг (нейрон мережі) відповідає операції скалярного добутку свого вектора $w_{i}$ на набір вхідних змінних $x$. Таким чином, на виході прихованого шару ми отримаємо 3 нові змінні:

$$y_{1} = w_{1}^{T}x$$
$$y_{2} = w_{2}^{T}x$$
$$y_{3} = w_{3}^{T}x$$

&emsp;&emsp;Ми можемо об'єднати всі вектори $w$ в одну матрицю $W$ і записати в більш компактній формі:

$$y = Wx$$

&emsp;&emsp;В останньому нейроні (помаранчевий круг) буде навчений ще один вектор $z$, при домноженні на який ми отримаємо фінальний результат:

$$output = z^{T}y$$

&emsp;&emsp;Однак зверніть увагу, що $output = z^{T}y = z^{T}Wx = Bx$, де $B = z^{T}W$. Виходить, що не було сенсу вивчати матрицю параметрів $W$ і окремо вектор $z$, якщо можна було відразу вивчити вектор $B$, що призвело б нас до початкової форми моделі. Щоб додати ще один шар мережі був сенс до його виходу (вектор $y$) необхідно застосувати якесь нелінійне перетворення, яке не дозволить рівність $B = z^{T}W$. Таке перетворення називається функцією активації нейрона.

&emsp;&emsp;Найпопулярнішими функціями активації є сигмоїд та ReLU (rectified linear unit), які мають такий вигляд:

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.where(x > 0, x, 0)

x = np.linspace(-5, 5, 100)

plt.figure(figsize=(6, 3))
plt.subplot(121)
plt.plot(x, sigmoid(x))
plt.title('Sigmoid function')
plt.subplot(122)
plt.plot(x, relu(x))
plt.title('ReLU function')

plt.show()

&emsp;&emsp;Ускладнимо архітектуру нашої моделі, додавши додатковий прихований шар нейронів

In [ ]:
# фіксація випадкового стану
np.random.seed(1)
tf.random.set_seed(1)

model = Sequential()  # ініціалізація порожньої моделі
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(
    optimizer=SGD(learning_rate=0.03),  # алгоритм оптимізації та швидкість навчання
    loss='binary_crossentropy',  # функція втрат (та сама, що в логістичної регресії)
    metrics=['accuracy']  # додаткова метрика якості моделі
)

history = model.fit(x=X, y=y, batch_size=8, epochs=150)

In [ ]:
plot_decision_regions(X, y, clf=model)
plt.show()

&emsp;&emsp;У загальному випадку нейронна мережа може мати довільну кількість як прихованих шарів, так і нейронів у кожному шарі мережі. Для завдання класифікації за наявності кількох класів нам знадобиться кілька виходів моделі - по одному нейрону на кожен клас.

![](neural_net.jpg)

&emsp;&emsp;Для прикладу побудуємо модель класифікації ірисів, використовуючи всі 3 класи

In [ ]:
import pandas as pd

iris = datasets.load_iris()
X, y = iris.data, iris.target
y = pd.get_dummies(y).values  # цільову змінну представимо у вигляді one-hot вектора

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(type(X))

In [ ]:
y

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

mdl = Sequential()
mdl.add(Dense(units=3, activation='relu'))

# На виході моделі ми хочемо отримати вектор, що є розподілом ймовірностей для 3х класів.
# Це означає, що кількість нейронів має бути 3 і нам необхідно підібрати таку функцію активації, щоб
# розподіл вважалося допустимим (всі елементи були невід'ємними та у сумі давали 1). Хорошим кандидатом
#у такому разі є функція softmax
mdl.add(Dense(units=3, activation='softmax'))

&emsp;&emsp;Оскільки у нас тепер 3 класи, нам буде потрібна інша функція втрат. Зазвичай для класифікаційних завдань добре підходить категоріальна крос-ентропія як розширення логістичної функції втрат. Функція має таку формулу:

$$Loss = -\sum_{i=1}^{n}{\sum_{j=1}^{k}{y_{ij}log(p_{ij})})}$$

&emsp;&emsp;Тут $k$ - кількість класів. Для двох класів формула спрощується до вже відомої функції логістичної втрати.

In [ ]:
mdl.compile(
    optimizer=SGD(learning_rate=0.005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = mdl.fit(
    x=X_train, 
    y=y_train,
    batch_size=8,
    epochs=200,
    validation_data=(X_test, y_test)
)

In [ ]:
plot_model_history(history)

# Основи работи із зображеннями

&emsp;&emsp;Згортковою нейронною мережею називається така, яка включає операцію згортки. Такого типу мережі - це основний алгоритм для роботи із зображеннями та багатьма завданнями обробки сигналів.

&emsp;&emsp;Почнемо з того, що розберемося, що собою являє картинка. По-суті, в очах комп'ютера, картинка - це тривимірний масив, який у перших двох осях містить координати пікселя на зображенні, а в осі, що залишилася, його яскравість. Якщо картинка кольорова, то яскравість задається трьома числами: яскравістю червоного, зеленого та блакитного кольорів. Якщо картинка чорно-біла, то достатньо одного числа.

&emsp;&emsp;Яскравість зазвичай знаходиться в інтервалі від 0 до 255, де 0 відповідає повній відсутності кольору (у чорно-білому випадку це чорний піксель), а 255 - максимальної яскравості (ідеально білого кольору). Всі проміжні значення є відтінками сірого кольору.

&emsp;&emsp;Наприклад, ми можемо намалювати кольорову лінію на чорному фоні, використовуючи numpy таким чином:

In [ ]:
# створимо тривимірний масив із нулів, що відповідає чорному фону
img = np.zeros((28, 28, 3), dtype=np.int32)
img[:, 14, :] = [124, 50, 255]  # додамо лінію довільного кольору як стовпець

plt.imshow(img)
plt.show()

&emsp;&emsp;У більшості випадків при роботі з зображеннями об'єм даних занадто великий, щоб поміститися в оперативну пам'ять комп'ютера. Тому прийнято зчитувати картинки порційно (батч) прямо з диска. При відповідній структурі папок з картинками ми можемо скористатися вбудованим класом keras, який дозволяє зчитувати картинки та передавати їх моделі для навчання.

&emsp;&emsp;Для завдання класифікації структура папки має бути така:

&emsp;&emsp;1. Навчальні та тестові дані повинні знаходитись у різних папках (наприклад, train/test)

&emsp;&emsp;2. У кожній папці має бути стільки папок, скільки є класів у завданні. Назва папки повинна відповідати порядковому номеру класу в дослідженні і в цій папці повинні знаходитись лише картинки, що належать цьому класу.

&emsp;&emsp;Розглянемо приклад із розпізнаванням рукописних цифр. По суті, це завдання класифікації: на вхід подається картинка з якоюсь цифрою, на виході ми хочемо отримати число від 0 од 9, яке відповідає написаній на малюнку цифрі. Розбивши дані на 2 папки (testing/training) у кожній створимо по 10 папок із назвами від 0 до 9, як це зроблено в папці з даними MNIST.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Ініціалізуємо генератори картинок для навчального та тестового набору. При роботі із зображеннями прийнято
# нормувати яскравість так, щоб вона була в інтервалі від 0 до 1, тому передамо параметр rescale 1/255,
# що відповідає поділу на 255 кожного пікселя оригінальної картинки.
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# створення генератора картинок для навчального набору
train_gen = train_datagen.flow_from_directory(
    directory='./Data/MNIST/training',  # папка, звідки читати картинки
    target_size=(28, 28),  # розмір картинки (якщо оригінальне зображення не відповідає, то розмір буде змінено)
    color_mode="grayscale",  # колірна гама (у нашому випадку картинки чорно-білі)
    batch_size=8,  # розмір батча (скільки картинок зчитувати за один раз)
    class_mode="categorical",  # тип завдання (класи будуть взяті з назви папок, у яких лежать картинки)
    shuffle=True,  # чи перемішувати картинки при зчитуванні або читати по порядку
    seed=42  # фіксація випадкового стану
)

&emsp;&emsp;Проходячи циклом по створеному нами генератору ми будемо на кожному кроці отримувати по 8 картинок і по 8 класів, які з ними асоціюються у форматі one-hot вектора. Такий цикл може тривати нескінченно, тому виходити з нього завжди потрібно вручну.

In [ ]:
for img, label in train_gen:    
    # цикл, в якому додамо картинку на графік
    for i in range(img.shape[0]):        
        image_label = np.argmax(label[i])  # отримаємо клас картинки з one-hot вектора
        plt.subplot(2, 4, i+1)
        # для чорно-білої картинки третя вісь є зайвою, тому позбудемося її і оголосимо гаму як "gray"
        plt.imshow(img[i].reshape(28, 28), cmap='gray')
        plt.title(f'This is {image_label}')    
    print(label)
    break
    
plt.show()

# Збагачення даних (аугментація)

&emsp;&emsp;У широкому значенні слова аугментація - це будь-який спосіб додати спостереження або змінні до даних. Наприклад, якщо завдання кредитного скорингу є спосіб отримати доступ до даних бюро кредитних історій і додати цю інформацію в модель, це аугментація.

&emsp;&emsp;При роботі із зображеннями або сигналами під аугментацією зазвичай розуміють процес створення нових спостережень шляхом зміни існуючих. Справа в тому, що для людського ока яскравість картинки, положення об'єкта і ще багато деталей зображення часто не є критичними при розпізнаванні класу, однак через те, як машини бачать зображення, для комп'ютера це може бути далеко не настільки очевидно.

&emsp;&emsp;Наприклад, стоїть завдання знайти картинки, на яких присутня тільки вертикальна лінія:

In [ ]:
np.random.seed(1)

for i in range(8):    
    img = np.zeros((28, 28, 3), dtype=np.int32)  # ініціалізація чорної картинки
    img += np.random.randint(0, 100, size=1)[0]  # довільна зміна яскравості фону
    line_position = np.random.randint(0, 28, size=1)[0]  # довільний вибір положення лінії
    line_color = np.random.randint(0, 255, 3)  # довільний колір лінії
    
    img[:, line_position, :] = line_color  # додавання лінії на фон
    
    plt.subplot(2, 4, i + 1)
    plt.imshow(img)
    
plt.show()
    
    

&emsp;&emsp;Людина розуміє, що всі картинки вище належать до одного класу: картинки, на яких є вертикальна лінія. Однак для машини це всі різні картинки. Такі перетворення допомагають донести інформацію про несуттєві атрибути зображення. У цьому випадку несуттєвим є положення лінії, колір фону та самої лінії.

&emsp;&emsp;Можемо перевизначити наш генератор з найбільш популярними типами аугментації:

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=5, # випадкові поворот картинки в інтервалі + - 5 градусів
    width_shift_range=2, # випадковий зсув картинки по горизонталі на +- 2 пікселя
    height_shift_range=2, # випадковий зсув картинки по вертикалі на +- 2 пікселя
    brightness_range=(0.90, 1.1), # інтервал випадкової зміни яскравості (+- 10%)
    zoom_range=(0.95, 1.05), # інтервал випадкового наближення/віддалення (+- 5%)
    horizontal_flip=False, # чи робити випадково дзеркальне відображення картинки по горизонталі
    vertical_flip=False, # чи робити випадково дзеркальне відображення картинки по вертикалі
    preprocessing_function=None, # довільна функція, яка застосовує перетворення користувача
    rescale=1./255 # приведення яскравості до інтервалу [0, 1]
)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    directory='./Data/MNIST/training', # папка, звідки читати картинки
    target_size=(28, 28), # розмір картинки (якщо оригінальна картинка не відповідає розміру, то розмір буде змінено)
    color_mode="grayscale", # кольорова гама (у нашому випадку картинки чорно-білі)
    batch_size=8, # розмір батча (скільки картинок зчитувати за один раз)
    class_mode="categorical", # тип завдання (класи будуть взяті з назви папок, в яких лежать картинки)
    shuffle=True, # чи перемішувати картинки при зчитуванні або читати по порядку
    seed=42 # фіксація випадкового стану
)

&emsp;&emsp;З огляду на те, що кожна картинка з додаванням випадкових модифікацій, стає практично унікальною, а генерувати такі картинки ми можемо нескінченно, то, по суті, ми отримуємо дані нескінченного розміру. Однак на практиці додаткової інформації з ходом навчання стає все менше і модель зупиняється на якийсь епосі після погіршення помилки на тесті.

&emsp;&emsp;При аугментації дуже важливо розуміти, які зміни дійсно є некритичними і дозволять збагатити дані, а які ні. Наприклад, якщо завдання варто зрозуміти тип хмари, то картинку можна перевернути ногами вгору і від цього хмара залишиться хмарою. Але якщо так само зробити з цифрами, то 6 перетвориться на 9, що призведе до небажаної зміни класу.

&emsp;&emsp;Створимо генератор тестових зображень так само:

In [ ]:
test_gen = test_datagen.flow_from_directory(
    directory='./Data/MNIST/testing', # папка, звідки читати картинки
    target_size=(28, 28), # розмір картинки (якщо оригінальна картинка не відповідає по розміру, то розмір буде змінено)
    color_mode="grayscale", # кольорова гама (у нашому випадку картинки чорно-білі)
    batch_size=8, # розмір батча (скільки картинок зчитувати за один раз)
    class_mode="categorical", # тип завдання (класи будуть взяті з назви папок, в яких лежать картинки)
    shuffle=True, # чи перемішувати картинки при зчитуванні або читати по порядку
    seed=42 # фіксація випадкового стану
)

# Операція згортки

&emsp;&emsp;Картинка - це приклад неструктурованих даних, тобто таких, у яких змінні не знаходяться в одній зручній таблиці. Для того, щоб якийсь алгоритм міг працювати з такими даними (наприклад, вирішувати задачу класифікації зображень), нам потрібно якимось чином привести дані з неструктурованого формату до структурованого.

&emsp;&emsp;Повернемося до прикладу розпізнавання картинки, де зображена пряма лінія. Ми розуміємо шаблон, за яким картинку з вертикальною лінією можна відрізнити від картинки з горизонтальною. Передати наші знання машині можна шляхом визначення маски, скажімо, наступного формату:

In [ ]:
vert_mask = -np.ones((3, 3), dtype=np.int32)
vert_mask[:, 1]  = 2
print(vert_mask)

&emsp;&emsp;Тобто, якщо десь на картинці буде ділянка, схожа на маску в тому сенсі, що з трьох рядів пікселів тільки центральний заповнений якимось кольором, то швидше за все ця картинка містить вертикальну лінію.

&emsp;&emsp;Застосовується маска таким чином:

&emsp;&emsp;1. Накладається на якусь ділянку зображення

&emsp;&emsp;2. Усі елементи маски перемножуються зі значеннями яскравості відповідних пікселів і підсумовуються (така операція називається згорткою)

In [ ]:
# створимо картинку з вертикальною лінією
img = np.zeros((4, 4), dtype=np.float64)
img[:, 2] = 1

plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
img 

&emsp;&emsp;Оскільки ми не знаємо, куди потрібно прикласти маску, спробуємо на всі можливі позиції на картинці зі змішенням в 1 піксель:

In [ ]:
def convolve(arr1, arr2):
    "Визначимо операцію згортки для двох масивів"
    return np.sum(arr1 * arr2)

def apply_mask(image, mask):
    "Визначимо функцію, яка застосовує маску до картинки на всіх можливих позиціях"
    
    # визначимо скільки є можливих положень маски по горизонталі та вертикалі
    x_positions = image.shape[0] - mask.shape[0] + 1
    y_positions = image.shape[1] - mask.shape[1] + 1
    
    # ініціалізуємо масив, в якому зберігатимемо результати згортки
    res = np.empty((x_positions, y_positions))
    
    # застосуємо маску до всіх можливих положень картинки та збережемо результати
    for i in range(x_positions):
        for j in range(y_positions):
            res[i, j] = convolve(mask, image[i:i+mask.shape[0], j:j+mask.shape[1]])
    
    return res

print(apply_mask(img, vert_mask))

&emsp;&emsp;Як і очікувалося, там де маска "знайшла" свій шаблон, значення згортки дорівнює 6, а там, де не "знайшла", то значення рівне -3. Визначимо ще маску, яка шукатиме горизонтальну лінію і спробуємо вирішити задачу розрізнення картинок з горизонтальною і вертикальною.

In [ ]:
horizontal_mask = -np.ones((3, 3), dtype=np.int32)
horizontal_mask[1, :] = 2

print(horizontal_mask)

In [ ]:
print(apply_mask(img, horizontal_mask))

&emsp;&emsp;Тепер згенеруємо 2 набори даних (картинок з різними лініями різного відтінку сірого кольору):

In [ ]:
num_images = 100

data, labels = [], []
for i in range(num_images):
    img = np.zeros((4, 4), dtype=np.int32) # створимо чорний фон
    is_vertical = np.random.randint(0, 2, size=1)[0] # виберемо тип лінії випадково
    line_position = np.random.randint(1, 3, size=1)[0] # виберемо випадково положення лінії
    brightness = np.random.randint(30, 255, size=4) # виберемо яскравість лінії
    
    # заповнимо лінію на зображенні
    if is_vertical == 1:
        img[:, line_position] = brightness
        labels.append(1)
    else:
        img[line_position, :] = brightness
        labels.append(0)
        
    data.append(img)

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(data[i], cmap='gray')
    
plt.show()

&emsp;&emsp;Тепер розглянемо результат застосування обох наших масок до першої картинки:

In [ ]:
horizontal_res = apply_mask(data[0], horizontal_mask)
vertical_res = apply_mask(data[0], vert_mask)

plt.imshow(data[0], cmap='gray')
plt.show()

print('Горизонтальна маска:')
print(horizontal_res)
print('Вертикальна маска:')
print(vertical_res)

&emsp;&emsp;З результатів видно, що шаблон горизонтальної маски виконується частково, тому отримані для неї результати згортки менші, ніж для вертикальної. Однак наша початкова проблема все ще не вирішена: нам потрібно було отримати таблицю змінних з картинки, а ми отримали іншу картинку просто меншого розміру.

&emsp;&emsp;Перевести результати застосування маски в змінні табличного виду можна двома способами:

&emsp;&emsp;1. Розглядати усі результати застосування маски як змінні. Тобто ми просто зробимо отримані масиви "плоськими", після чого продовжимо працювати з 8 змінними.

&emsp;&emsp;2. Застосувати операцію **пулінгу**. Ця операція полягає у агрегації результатів застосування маски. Наприклад, у нашому випадку ми могли б взяти максимальний результат згортки з чотирьох або середній.

&emsp;&emsp;Нарешті, створимо наші змінні як максимальний результат згортки для кожної маски:

In [ ]:
features = []
for img in data:
    features.append([
        np.max(apply_mask(img, horizontal_mask)),
        np.max(apply_mask(img, vert_mask))
    ])
    
features = pd.DataFrame(features, columns=['horizontal', 'vertical'])
features.head()

&emsp;&emsp;Тепер ми можемо скористатися будь-яким алгоритмом класифікації, щоб легко відрізнити одні картинки від інших:

In [ ]:
plt.scatter(features['horizontal'], features['vertical'], c=labels)
plt.show()

# Згорткові нейронні мережі

&emsp;&emsp;Проблема в тому, що на реальних завдачах створити досить хороші маски зазвичай немає можливості. Але виявляється, що їх можна вивчити із самих даних. Задамо одну маску як таблицю параметрів:

$$mask = \begin{pmatrix} w_{1} w_{2} w_{3} \\ w_{4} w_{5} w_{6} \\ w_{7} w_{8} w_{9} \end{pmatrix}$$

&emsp;&emsp;Початкові значення цих параметрів можна згенерувати випадково. Застосовуючи таку маску до зображення і знаючи помилку моделі, ми можемо отримати похідну функції втрат кожного елемента маски, який тепер представлений параметром. При цьому ми можемо ініціалізувати довільну кількість масок, розширюючи тим самим множину шаблонів, які ми шукатимемо на зображенні.

&emsp;&emsp;На практиці згорткові нейронні мережі виглядають як послідовне застосування набору масок (заданих у вигляді параметрів, що навчаються), активацій (як правило ReLU) і операцій пулінгу. Тобто ми можемо визначити маски, які застосовуватимуться до результатів застосування інших масок. Так, у прикладі вище ми після застосування маски отримали з картинки розміру 4х4 картинку розміру 2х2, що дозволяє ще раз застосувати до такої маски відповідного розміру.

&emsp;&emsp;У самому кінці моделі всі параметри зазвичай призводять до вигляду одновимірного масиву і нейронна мережа перетворюється на багатошаровий перцептрон.

In [ ]:
# визначимо архітектуру мережі для класифікації рукописних цифр
from keras.layers import Conv2D, MaxPool2D, Flatten

np.random.seed(1)
tf.random.set_seed(1)

model = Sequential()  # ініціалізація моделі
# додамо перший шар, який складатиметься із застосування масок, що навчаються
model.add(Conv2D(
    filters=8,  # кількість різних масок, які ми хочемо навчити
    kernel_size=4,  # розмір однієї маски (4х4 пікселя)
    activation='relu',  # активація після застосування маски
    input_shape=(28, 28, 1)  # розмір картинки, яка буде подана на вхід
))
# додамо операцію пулінга, при цьому вкажемо розмір вікна: брати максимум з розмірів розміру 2х2
model.add(MaxPool2D(pool_size=(2, 2)))
# додамо ще один шар масок та пулінг
model.add(Conv2D(filters=16, kernel_size=4, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# наведемо мережу до одномірного формату і додамо шар нейронів, що навчається
model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=10, activation='softmax'))  # останній шар з активацією для завдання класифікації

&emsp;&emsp;Тепер ми можемо переглянути структуру нашої мережі:

In [ ]:
model.summary()

&emsp;&emsp;Подальший процес навчання складається з тих самих кроків. Для початку скомпілюємо модель

In [ ]:
model.compile(
    optimizer=SGD(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

&emsp;&emsp;Навчання моделі з використанням генераторів даних виглядає так:

In [ ]:
history = model.fit(
    x=train_gen,
    validation_data=test_gen,
    epochs=3,
    workers=8,
    steps_per_epoch=len(train_gen),
    validation_steps=len(test_gen)
)

# mnist_fashion

Ми будемо використовувати набір даних [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), який містить 70 000 зображень у відтінках сірого в 10 категоріях. На зображеннях показано окремі предмети одягу з низькою роздільною здатністю (28 на 28 пікселів), як показано тут:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST призначений як додаткова заміна класичного набору даних [MNIST](http://yann.lecun.com/exdb/mnist/), який часто використовується як «Hello, World» програм машинного навчання для комп’ютерного зору. Набір даних MNIST містить зображення рукописних цифр (0, 1, 2 тощо) у форматі, ідентичному формату предметів одягу, які ви тут використовуватимете.

Ми використовуємо Fashion MNIST для різноманітності, а також тому, що це дещо складніша проблема, ніж звичайний MNIST. Обидва набори даних є відносно невеликими та використовуються для перевірки того, що алгоритм працює належним чином. Вони є хорошою відправною точкою для тестування та налагодження коду.

Тут 60 000 зображень використовуються для навчання мережі та 10 000 зображень для оцінки того, наскільки точно мережа навчилася класифікувати зображення. Ви можете отримати доступ до Fashion MNIST безпосередньо з TensorFlow. Імпортуйємо та завантажуємо дані Fashion MNIST безпосередньо з TensorFlow:


In [ ]:
import tensorflow
from tensorflow.keras.datasets.fashion_mnist import load_data
#fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = load_data()

Завантаження набору даних повертає чотири масиви NumPy:

* Масиви `train_images` і `train_labels` є *навчальним набором* — даними, які модель використовує для навчання.
* Модель перевіряється на основі *тестового набору*, масивів `test_images` і `test_labels`.

Зображення являють собою масиви 28x28 NumPy із значеннями пікселів у діапазоні від 0 до 255. *Мітки* — це масив цілих чисел у діапазоні від 0 до 9. Вони відповідають *класу* одягу, який представляє зображення:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Кожне зображення зіставляється з однією міткою. Оскільки *назви класів* не входять до набору даних, збережмо їх тут, щоб використовувати пізніше під час побудови зображень:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### Аналіз даних

Давайте вивчимо формат набору даних перед навчанням моделі. Нижче показано, що в навчальному наборі є 60 000 зображень, кожне зображення представлене як 28 x 28 пікселів:

In [ ]:
print(train_images.shape)
len(train_labels)

In [ ]:
train_labels

In [ ]:
test_images.shape

## Попередня обробка даних

Перед навчанням мережі дані необхідно попередньо обробити. Якщо ми перевіримо перше зображення в навчальному наборі, ми побачемо, що значення пікселів знаходяться в діапазоні від 0 до 255:

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.imshow(train_images[0], cmap='gray')

Змінемо ці значення в діапазоні від 0 до 1, перш ніж передати їх у модель нейронної мережі. Для цього розділемо значення на 255. Важливо, щоб *навчальний набір* і *тестовий набір* були попередньо оброблені однаково:

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Щоб переконатися, що дані мають правильний формат і що ми готові створювати та навчати мережу, давайте відобразимо перші 25 зображень із навчального набору та відобразимо назву класу під кожним зображенням.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

### Побудуємо модель

Побудова нейронної мережі вимагає налаштування рівнів моделі, а потім компіляції моделі.

#### Налаштування шарів

Основним будівельним блоком нейронної мережі є *шар*. Шари витягують представлення з даних, що вводяться в них. 

Більшість нейронних мереж складається з об’єднання простих шарів. Більшість шарів, наприклад `tf.keras.layers.Dense`, мають параметри, які вивчаються під час навчання.

In [ ]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

ModuleNotFoundError: No module named 'tensorflow'

**Перший** шар у цій мережі, `tf.keras.layers.Flatten`, перетворює формат зображень із двовимірного масиву (28 на 28 пікселів) на одновимірний масив (28 *). 28 = 784 пікселів). Цей шар не має параметрів для вивчення, він лише переформатує дані.

Після вирівнювання пікселів мережа складається з послідовності двох шарів `tf.keras.layers.Dense`. Це щільно з’єднані або повністю з’єднані нейронні шари. Перший «щільний» шар має 128 вузлів (або нейронів). Другий (і останній) шар повертає масив довжиною 10. Кожен вузол містить оцінку, яка вказує, що поточне зображення належить до одного з 10 класів.

### Скомпілюйте модель

Перед тим, як модель буде готова до навчання, їй потрібно ще кілька налаштувань. Вони додаються на етапі *компіляції* моделі:

* *Функція втрат* — вимірює, наскільки точна модель під час навчання. Ми хочемо мінімізувати цю функцію, щоб «спрямувати» модель у правильному напрямку.
* *Оптимізатор* — таким чином модель оновлюється на основі даних, які вона бачить, і її функції втрат.
* *Метриики* — використовується для моніторингу етапів навчання та тестування. У наступному прикладі використовується *точність*, частка правильно класифікованих зображень.

In [ ]:
import tensorflow as tf
model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

ModuleNotFoundError: No module named 'tensorflow'

### Навчання моделі

Навчання моделі нейронної мережі вимагає наступних кроків:

1. Подайти навчальні дані в модель. У цьому прикладі навчальні дані містяться в масивах train_images і train_labels.
2. Модель вчиться асоціювати зображення та мітки.
3. Ми просимо модель зробити прогнози щодо тестового набору — у цьому прикладі масиву test_images.
4. Переконатися, що передбачення збігаються з мітками з масиву test_labels.

Щоб розпочати навчання, викличемо метод `model.fit` — так він називається, тому що він «підлаштовує» модель під навчальні дані:

In [ ]:
model.fit(train_images, train_labels, epochs=10)

#### Оцінка точность

Далі порівняємо, як модель працює на тестовому наборі даних:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Виявляється, точність тестового набору даних трохи менша, ніж точність навчального набору даних. Цей розрив між точністю навчання та точністю тесту означає *перенавчання*. Перенавчання відбувається, коли модель машинного навчання працює гірше на нових, раніше невидимих вхідних даних, ніж на навчальних даних. Перенавчена модель «запам’ятовує» шум і деталі в навчальному наборі даних до точки, коли це негативно впливає на ефективність моделі на нових даних.

### Нормалізація ваг

«Проста модель» у цьому контексті — це модель, у якій розподіл значень параметрів має меншу ентропію (або модель із зовсім меншою кількістю параметрів, як ми бачили  вище). Таким чином, звичайний спосіб пом’якшити перенавчання полягає в тому, щоб накласти обмеження на складність мережі, змушуючи її ваги приймати лише малі значення, що робить розподіл значень ваг більш «регулярним». Це називається «регуляризацією ваги», і це робиться шляхом додавання до функції втрат мережі вартості, пов’язаної із великими вагами. Ця вартість має два варіанти:

* Регулярізація L1, де додана вартість пропорційна абсолютному значенню вагових коефіцієнтів (тобто тому, що називається «нормою L1» ваг).
* Регулярізація L2, де додана вартість пропорційна квадрату значення вагових коефіцієнтів (тобто тому, що називається квадратом «норми L2» ваг). Регуляризація L2 також називається спадом ваги в контексті нейронних мереж. 

Регулярізація L1 підштовхує ваги до точного нуля, заохочуючи розріджену модель. Регулярізація рівня L2 штрафуватиме параметри ваг, не роблячи їх розрідженими, оскільки для малих ваг штраф буде нульовим. одна з причин, чому L2 є більш поширеною.

У tf.keras регулярізація ваги додається шляхом передачі екземплярів регуляризатора ваги до шарів як аргументів. Давайте зараз додамо регулярізацію ваги L2.

#### Застосування L2 регуляризації

In [ ]:
from tensorflow.keras import regularizers
model_l2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dense(10)
])
model_l2.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model_l2.fit(train_images, train_labels, epochs=10)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
test_loss_l2, test_acc_l2 = model_l2.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc_l2)

Як можна побачити вище, переонавчання певною мірою усунуто з моделі, але за рахунок продуктивності.

#### Додавання вилученя
Dropout є одним із найефективніших і найчастіше використовуваних методів регуляризації для нейронних мереж, розроблених Хінтоном та його студентами з Університету Торонто.

Інтуїтивно зрозуміле пояснення вилучення полягає в тому, що оскільки окремі вузли в мережі не можуть покладатися на вихідні дані інших, кожен вузол повинен виводити функції, які є корисними самостійно.

Вилучення, застосоване до шару, складається з випадкового «випадання» (тобто встановленого на нуль) ряду вихідних характеристик шару під час навчання. Скажімо, даний рівень зазвичай повертає вектор [0,2, 0,5, 1,3, 0,8, 1,1] для даного вхідного зразка під час навчання; після застосування вилучення цей вектор матиме кілька нульових записів, розподілених випадковим чином, наприклад. [0, 0,5, 1,3, 0, 1,1].

У tf.keras ви можете запровадити вилучення в мережі через рівень Dropout, який застосовується до виводу шару безпосередньо перед цим.

Давайте додамо два шари Dropout у нашу мережу, щоб побачити, наскільки добре вони справляються зі зменшенням перенавчання:

In [ ]:
from tensorflow.keras import layers
model_dropout = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    keras.layers.Dense(10)
])
model_dropout.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model_dropout.fit(train_images, train_labels, epochs=10)

In [ ]:
test_loss_dropout, test_acc_dropout = model_dropout.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc_dropout)

Ми застосували відсіювання на одному шарі з 128 вузлів, зробивши вихід 30% вузлів нулями, ми змогли зменшити перенавчання більшою мірою, ніж регулярізація l2

#### Комбінація L2 + вилучення

In [ ]:
from tensorflow.keras import regularizers
model_l2_dropout = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    keras.layers.Dense(10)
])
model_l2_dropout.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model_l2_dropout.fit(train_images, train_labels, epochs=10)

In [ ]:
test_loss_l2_dropout, test_acc_l2_dropout = model_l2_dropout.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc_l2_dropout)

### Робимо прогноз

Навчену модель ми можемо використовувати, щоб робити прогнози щодо деяких зображень.

In [ ]:
probability_model = tf.keras.Sequential(
    [model, tf.keras.layers.Softmax()]
)
predictions = probability_model.predict(test_images)

Тут модель передбачила мітку для кожного зображення в наборі для тестування. Давайте подивимося на перший прогноз:

In [ ]:
predictions[0]

In [ ]:
test_labels[0]

In [ ]:
plt.imshow(test_images[0], cmap='gray')

Побудуємо це на графіку, щоб переглянути повний набір прогнозів із 10 класів.

In [ ]:
import numpy as np

def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
    color = 'blue'
    else:
    color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

### Перевіка прогнозу
Наввчену модель ми можемо використовувати, щоб робити прогнози щодо деяких зображень.

Давайте подивимося на 0-е зображення, прогнози та масив прогнозів. Мітки правильного прогнозу сині, а неправильні – червоного. Число дає відсоток (зі 100) для прогнозованої мітки.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Давайте нанесемо кілька зображень з їх передбаченнями. Зауважемо, що модель може помилятися, навіть якщо вона дуже впевнена.

In [ ]:
# Побудуємо перші X тестових зображень, їхні передбачені мітки та справжні мітки.
# Правильні передбачення пофарбуємо синім кольором, а неправильні – червоним.
print(class_names)
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

Нарешті, використємо навчену модель, щоб спрогнозувати одне зображення.

In [ ]:
# Візьмемо зображення з тестового набору даних.
img = test_images[1]

print(img.shape)

In [ ]:
# Додаймо зображення до групи, де воно є єдиним членом.
img = (np.expand_dims(img,0))

print(img.shape)

In [ ]:
predictions_single = probability_model.predict(img)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

# Нейронні мережі в SkLearn

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions

X, y = make_moons(n_samples=100, noise=0.25, random_state=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
random_state=42)
mlp = MLPClassifier(solver='lbfgs', random_state=0).fit(X_train, y_train)
plot_decision_regions(X, y, clf=mlp)

In [ ]:
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[10]).fit(X_train, y_train)
plot_decision_regions(X, y, clf=mlp)

In [ ]:
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[10, 10]).fit(X_train, y_train)
plot_decision_regions(X, y, clf=mlp)

In [ ]:
mlp = MLPClassifier(
    solver='lbfgs', activation='tanh', random_state=0, hidden_layer_sizes=[10, 10]
).fit(X_train, y_train)
plot_decision_regions(X, y, clf=mlp)

In [ ]:
from matplotlib import pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(20, 8))

for axx, n_hidden_nodes in zip(axes, [10, 100]):
    for ax, alpha in zip(axx, [0.0001, 0.01, 0.1, 1]):
        mlp = MLPClassifier(
            solver='lbfgs', 
            random_state=0,
            hidden_layer_sizes=[n_hidden_nodes, n_hidden_nodes], 
            alpha=alpha
        )
        mlp.fit(X_train, y_train)
        plot_decision_regions(X, y, clf=mlp, ax=ax)
        ax.set_title("n_hidden=[{}, {}]\nalpha={:.4f}".format(n_hidden_nodes, n_hidden_nodes, alpha))